Create a DataFrame from an RDD through reflection method

Import findspark and initiate.
Then import pyspark

In [ ]:
import findspark
findspark.init('/usr/local/spark')
import pyspark

Start SparkSession

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL example").getOrCreate()

In [ ]:
#import collections

In [ ]:
#collections.namedtuple

In [ ]:
from pyspark.sql import Row

In [ ]:
r1 = Row(name="John",Age=28)
print(r1.name)
print(r1.Age)
print(r1)

In [ ]:
sc = spark.sparkContext

Create an RDD from the structured text file

In [ ]:
clines = sc.textFile("customers.txt")

Transform this RDD of text lines to an RDD of "Row"s with each Row having the 5 fields of the text input lines.

In [ ]:
cfields = clines.map(lambda l: l.split("\t"))

In [ ]:
cfields.take(2)

In [ ]:
customers = cfields.map(lambda p: Row(cid=p[0],cname=p[1],ccity=p[2],cstate=p[3],czip=p[4]))

In [ ]:
customers.take(2)

In [ ]:
customers

Infer the schema, and register the DataFrame as a table (temp view).

In [ ]:
customerDF = spark.createDataFrame(customers)

In [ ]:
customerDF.printSchema()

In [ ]:
customerDF.show(5)

In [ ]:
customerDF.head()

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

In [ ]:
df_indexed=customerDF.withColumn("index", monotonically_increasing_id())

In [ ]:
df_indexed.printSchema()

In [ ]:
df_indexed.show(3)

In [ ]:
customerDF=df_indexed.filter(df_indexed.index!=0).drop("index")

In [ ]:
customerDF.show(2)

In [ ]:
customerDF.select("cname").show(5)

In [ ]:
customerDF.select(customerDF['cname'], customerDF['ccity']).show(2)

In [ ]:
customerDF.filter(customerDF['cstate'] == 'CA').select(customerDF['cname'],customerDF['cstate']).show(5)

In [ ]:
customerDF.groupBy("cstate").count().show()

In [ ]:
customerDF.show(20)

Create the temp view to be able to run SQL queries on the DataFrame

In [ ]:
customerDF.createOrReplaceTempView("customers")

In [ ]:
cStateCount50 = spark.sql("SELECT cstate, count(*) as sttcount FROM customers GROUP BY cstate HAVING sttcount>=50")

In [ ]:
type(cStateCount50)

In [ ]:
cStateCount50.show()

In [ ]:
cStateCount50.printSchema()

In [ ]:
pwd

In [ ]:
cStateCount50.write.save('mydata')

In [ ]:
cStateCount50.write.parquet('state_ct1.parquet')

In [ ]:
# Coalesce can reduce/merge the number of partitions to a few

In [ ]:
cStateCount50.coalesce(2).write.parquet('state_ct1.parquet')

In [ ]:
cStateCount50.coalesce(2).write.json('mydata.json')

In [ ]:
type(cStateCount50)

In [ ]:
spark.read.json('mydata.json').show()

In [ ]:
spark.read.parquet('state_ct1.parquet').show()

In [ ]:
x=sc.parallelize([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], numSlices=3)

In [ ]:
x.glom().collect()

In [ ]:
# Coalesce cannot increase the number of partitions/it will only merge the nearest 
# partitions, it doesn't shuffle the data across the partitions
x.coalesce(2).glom().collect()

In [ ]:
x.coalesce(4).glom().collect()

In [ ]:
x.repartition(2).glom().collect()

In [ ]:
x.repartition(5).glom().collect()

In [ ]:
data = [(i,i%2) for i in range(1,1000001)]

In [ ]:
rdd1 = spark.sparkContext.parallelize(data,8)

In [ ]:
df = rdd1.toDF(["number", "mod_value"])

In [ ]:
df.show(5)

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
repartitioned_df=df.repartition(16,"mod_value")

In [ ]:
repartitioned_df.rdd.getNumPartitions()

In [ ]:
coalesced_df = df.coalesce(4)

In [ ]:
coalesced_df.rdd.getNumPartitions()

In [ ]:
repartitioned_df.groupBy("mod_value").count().show()

In [ ]:
coalesced_df.groupBy("mod_value").count().show()

In [ ]:
spark

In [ ]:
# shuffling --> Achieve evenly distribution of the data in all the partitions

In [ ]:
data = [(i,) for i in range(1,13)]

In [ ]:
df = spark.createDataFrame(data, ['number']).repartition(4)

In [ ]:
df.show()

In [ ]:
df.withColumn("part_id", spark_partition_id()).show()

In [ ]:
from pyspark.sql.functions import spark_partition_id

In [ ]:
repart = df.repartition(3)

In [ ]:
repart.withColumn("part_id", spark_partition_id()).show()

In [ ]:
coalesced = df.coalesce(3)

In [ ]:
coalesced.withColumn("part_id", spark_partition_id()).show()

In [ ]:
[1,2,3] [4,5,6,7][8,9,10]

In [ ]:
[1,2,3,4,5,6,7] [8,9,10]

In [ ]:
[1,2,3][4,5,6]

In [ ]:
sc.stop()

In [ ]:
spark.stop()